In [1]:
import requests
import defs
import importlib
import pandas as pd


In [2]:
session = requests.Session()

In [3]:
ins_df = pd.read_pickle('instruments.pkl')
our_curr = ['EUR','USF','GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [13]:
#DEFINING FUNTIONS
def fetch_candles(pair_name, count, granularity):
    url = f'{defs.OANDA_URL}/instruments/{pair_name}/candles'
    params = dict(
        count = count,
        granularity = granularity,
        price = 'MBA'
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [16]:
#fetch_candles test
code, res = fetch_candles('EUR_USD', 10, 'H1')

code
#res

200

In [21]:
#DEFINING FUNTIONS Part II: turn into dataframe
def get_candles_df(json_response):
    prices = ['mid','bid','ask']
    ohlc = ['o','h','l','c']
    
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue

        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

In [22]:
code, res = fetch_candles('EUR_USD', 10, 'H1')
get_candles_df(res)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-06-12T10:00:00.000000000Z,1906,1.07529,1.07602,1.07491,1.07586,1.07522,1.07595,1.07483,1.07579,1.07536,1.07610,1.07499,1.07594
1,2024-06-12T11:00:00.000000000Z,2711,1.07587,1.07656,1.07573,1.07642,1.07580,1.07649,1.07566,1.07635,1.07594,1.07664,1.07580,1.07650
2,2024-06-12T12:00:00.000000000Z,9407,1.07643,1.08301,1.07621,1.08266,1.07636,1.08294,1.07614,1.08259,1.07650,1.08308,1.07628,1.08274
3,2024-06-12T13:00:00.000000000Z,8752,1.08263,1.08375,1.08190,1.08366,1.08256,1.08368,1.08182,1.08359,1.08270,1.08382,1.08197,1.08373
4,2024-06-12T14:00:00.000000000Z,5840,1.08368,1.08497,1.08176,1.08480,1.08360,1.08490,1.08169,1.08473,1.08375,1.08504,1.08183,1.08487
5,2024-06-12T15:00:00.000000000Z,3667,1.08481,1.08524,1.08418,1.08426,1.08474,1.08517,1.08411,1.08419,1.08488,1.08531,1.08425,1.08433
6,2024-06-12T16:00:00.000000000Z,2906,1.08424,1.08482,1.08416,1.08441,1.08417,1.08475,1.08409,1.08434,1.08431,1.08489,1.08423,1.08448
7,2024-06-12T17:00:00.000000000Z,2893,1.08440,1.08459,1.08362,1.08406,1.08433,1.08452,1.08355,1.08387,1.08447,1.08466,1.08369,1.08426
8,2024-06-12T18:00:00.000000000Z,20605,1.08404,1.08442,1.08076,1.08120,1.08382,1.08414,1.08068,1.08112,1.08425,1.08477,1.08084,1.08127


In [23]:
#DEFINING FUNTIONS Part III: turn dataframe into file
def save_file(candles_df, pair, granularity):
    candles_df.to_pickle(f'his_data/{pair}_{granularity}.pkl')

In [24]:
#DEFINING FUNTIONS Part IV: combination of all prev fxns
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(pair, 'ERROR')
        return
    df = get_candles_df(json_data)
    print(f'{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}')
    save_file(df, pair, granularity)

In [5]:
#ins_df.shape[0]
#ins_df.info()

ins_df.name.unique

<bound method Series.unique of 0     NZD_HKD
1     CAD_SGD
2     AUD_NZD
3     CAD_CHF
4     GBP_USD
       ...   
63    GBP_AUD
64    NZD_USD
65    USD_HKD
66    CHF_ZAR
67    EUR_USD
Name: name, Length: 68, dtype: object>

In [25]:
for p1 in our_curr:
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            #print(pair)
            create_data(pair, 'H1')

EUR_GBP loaded 3999 candles from 2023-10-20T07:00:00.000000000Z to 2024-06-12T23:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2023-10-20T07:00:00.000000000Z to 2024-06-12T23:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2023-10-20T07:00:00.000000000Z to 2024-06-12T23:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2023-10-20T09:00:00.000000000Z to 2024-06-12T23:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2023-10-20T07:00:00.000000000Z to 2024-06-12T23:00:00.000000000Z
GBP_JPY loaded 3999 candles from 2023-10-20T07:00:00.000000000Z to 2024-06-12T23:00:00.000000000Z
GBP_CHF loaded 3999 candles from 2023-10-20T07:00:00.000000000Z to 2024-06-12T23:00:00.000000000Z
GBP_NZD loaded 3999 candles from 2023-10-20T09:00:00.000000000Z to 2024-06-12T23:00:00.000000000Z
GBP_CAD loaded 3999 candles from 2023-10-20T07:00:00.000000000Z to 2024-06-12T23:00:00.000000000Z
CHF_JPY loaded 3999 candles from 2023-10-20T07:00:00.000000000Z to 2024-06-12T23:00:00.000000000Z
NZD_JPY loaded 3999 